In [1]:
from datasets.load import load_dataset
from evaluate import load
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

raw_datasets = load_dataset('iwslt2017', 'iwslt2017-zh-en')

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

Found cached dataset iwslt2017 (/Users/chen/.cache/huggingface/datasets/iwslt2017/iwslt2017-zh-en/1.0.0/03ce9110373117c6f6687719f49f269486a8cd49dcad2527993a316cd4b6ad49)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
en_zh_test = pd.DataFrame(raw_datasets['test']['translation'])
zh_truth = en_zh_test['zh'].tolist()
en_input = en_zh_test['en'].tolist()

In [3]:
from googletrans import Translator
translator = Translator()

In [9]:
from tqdm.auto import tqdm
zh_preds = []
for en in tqdm(en_input):
    zh_preds.append(translator.translate(en, src='en', dest='zh-cn').text)

  0%|          | 0/8549 [00:00<?, ?it/s]

ConnectTimeout: _ssl.c:975: The handshake operation timed out

In [10]:
len(zh_preds)

2069

In [11]:
for en in tqdm(en_input[2070:]):
    zh_preds.append(translator.translate(en, src='en', dest='zh-cn').text)

  0%|          | 0/6479 [00:00<?, ?it/s]

ConnectError: [Errno 65] No route to host

In [12]:
len(zh_preds)

7229

In [13]:
for en in tqdm(en_input[7230:]):
    zh_preds.append(translator.translate(en, src='en', dest='zh-cn').text)

  0%|          | 0/1319 [00:00<?, ?it/s]

In [14]:
len(zh_preds)

8548

In [19]:
idx = 2069
print(zh_preds[idx])
print(en_input[idx+1])

我要去这幅画中我最喜欢的部分之一，我真的要去裂缝。
I'm going to go to one of my favorite parts in this painting,  and I'm really going to get to the cracks.


In [21]:
zh_preds.insert(idx, translator.translate(en_input[idx], src='en', dest='zh-cn').text)

In [22]:
idx = 2069
print(zh_preds[idx])
print(en_input[idx])

你放大了。你真的进去了。
You zoom in. You really go in.


In [26]:
len(zh_preds)

8549

In [27]:
import pickle
with open('zh_preds_google.pkl', 'wb') as f:
    pickle.dump(zh_preds, f)

In [28]:
bleu = load('sacrebleu')
en_zh_bleu = bleu.compute(predictions=zh_preds, references=zh_truth, tokenize='zh')
print("----------------------------------------------")
print("English to Chinese: ", en_zh_bleu)

----------------------------------------------
English to Chinese:  {'score': 24.310291276147606, 'counts': [135451, 73330, 41284, 24538], 'totals': [233275, 224726, 216177, 207637], 'precisions': [58.06494480763048, 32.630848232959245, 19.097313775285993, 11.817739612882097], 'bp': 0.9506658471486413, 'sys_len': 233275, 'ref_len': 245077}
